In [1]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [2]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Dropout

Read CSV create, and clean dataframe

In [3]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Teams.csv")

mlb_teams

,yearID,lgID,G,G.1,W,L,WSWin,R,AB,H,...,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,1969,NL,162,81,93,69,N,691,5460,1411,...,22,141,485,665,59,3.53,38,7,42,Atlanta Braves
1,1969,AL,162,81,109,53,N,779,5518,1465,...,29,175,634,806,82,2.83,50,20,36,Baltimore Orioles
2,1969,AL,162,81,87,75,N,743,5494,1381,...,37,197,658,923,41,3.92,30,7,41,Boston Red Sox
3,1969,AL,163,81,71,91,N,528,5316,1221,...,29,88,516,929,54,3.54,25,9,39,California Angels
4,1969,AL,162,81,68,94,N,625,5450,1346,...,27,112,552,844,54,4.21,29,10,25,Chicago White Sox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2019,NL,162,81,91,71,N,764,5449,1336,...,24,210,561,1420,116,3.80,1,14,52,St. Louis Cardinals
1404,2019,AL,162,81,96,66,N,769,5628,1427,...,29,217,542,1493,94,3.65,0,12,46,Tampa Bay Rays
1405,2019,AL,162,81,78,84,N,810,5540,1374,...,24,223,534,1578,131,5.06,4,9,33,Texas Rangers
1406,2019,AL,162,81,67,95,N,726,5493,1299,...,21,247,509,1514,51,4.79,1,7,33,Toronto Blue Jays


In [4]:
#Get list of dataframe columns

mlb_teams.columns

Index(['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name'],
      dtype='object')

In [5]:
#Select columns for dataframe


mlb_df = mlb_teams[['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV']]
mlb_df.head(20)

,yearID,lgID,G,G.1,W,L,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV
0,1969,NL,162,81,93,69,N,691,5460,1411,195,22,141,485,665,59,3.53,38,7,42
1,1969,AL,162,81,109,53,N,779,5518,1465,234,29,175,634,806,82,2.83,50,20,36
2,1969,AL,162,81,87,75,N,743,5494,1381,234,37,197,658,923,41,3.92,30,7,41
3,1969,AL,163,81,71,91,N,528,5316,1221,151,29,88,516,929,54,3.54,25,9,39
4,1969,AL,162,81,68,94,N,625,5450,1346,210,27,112,552,844,54,4.21,29,10,25
5,1969,NL,163,82,92,70,N,720,5530,1400,215,40,142,559,928,30,3.34,58,22,27
6,1969,NL,163,81,89,73,N,798,5634,1558,224,42,171,474,1042,79,4.11,23,11,44
7,1969,AL,161,81,62,99,N,573,5365,1272,173,24,119,535,906,85,3.94,35,7,22
8,1969,AL,162,81,90,72,N,701,5441,1316,188,29,182,578,922,35,3.31,55,20,28
9,1969,NL,162,81,81,81,N,676,5348,1284,208,40,104,699,972,101,3.60,52,11,34


In [6]:
#Use getdummies to convert strings

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

data_bin.head()

,yearID,lgID,G,G.1,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
0,1969,NL,162,81,93,69,691,5460,1411,195,22,141,485,665,59,3.53,38,7,42,0
1,1969,AL,162,81,109,53,779,5518,1465,234,29,175,634,806,82,2.83,50,20,36,0
2,1969,AL,162,81,87,75,743,5494,1381,234,37,197,658,923,41,3.92,30,7,41,0
3,1969,AL,163,81,71,91,528,5316,1221,151,29,88,516,929,54,3.54,25,9,39,0
4,1969,AL,162,81,68,94,625,5450,1346,210,27,112,552,844,54,4.21,29,10,25,0


In [7]:
#Set X(data) and y(team)variables

data = data_bin.drop("lgID", axis=1)
outcome = data_bin["lgID"]
print(data.shape, outcome.shape)

(1408, 19) (1408,)


In [8]:
data_bin.count()

yearID     1408
lgID       1408
G          1408
G.1        1408
W          1408
L          1408
R          1408
AB         1408
H          1408
2B         1408
3B         1408
HR         1408
BB         1408
SO         1408
SB         1408
ERA        1408
CG         1408
SHO        1408
SV         1408
WSWin_Y    1408
dtype: int64

Preprocess data

In [9]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [10]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data, outcome, random_state=20)
X_train

,yearID,G,G.1,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
917,2003,161,80,66,95,642,5341,1317,262,24,124,489,1035,70,4.48,3,10,38,0
530,1990,162,81,65,97,682,5504,1376,263,26,162,473,1010,92,4.58,17,8,30,0
1328,2017,162,81,101,61,896,5611,1581,346,20,238,509,1087,98,4.12,1,9,45,1
713,1996,162,80,73,89,776,5665,1509,319,33,138,510,989,126,4.61,5,7,37,0
1215,2013,162,81,85,77,650,5449,1321,247,24,144,466,1214,115,3.94,7,10,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,2003,162,81,85,77,876,5672,1580,342,32,196,580,952,82,4.60,9,10,41,0
1247,2014,162,81,88,74,729,5545,1354,253,33,146,586,1104,83,3.22,7,13,31,0
271,1980,162,81,100,62,805,5585,1523,258,29,156,587,766,111,3.64,42,10,41,0
474,1987,162,81,90,72,783,5608,1458,274,32,205,511,1094,126,3.68,19,10,38,0


In [11]:
#Scale data using MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[0.68      , 0.95081967, 0.88571429, ..., 0.35714286, 0.48148148,
        0.        ],
       [0.42      , 0.96721311, 0.91428571, ..., 0.28571429, 0.33333333,
        0.        ],
       [0.96      , 0.96721311, 0.91428571, ..., 0.32142857, 0.61111111,
        1.        ],
       ...,
       [0.22      , 0.96721311, 0.91428571, ..., 0.35714286, 0.53703704,
        0.        ],
       [0.36      , 0.96721311, 0.91428571, ..., 0.35714286, 0.48148148,
        0.        ],
       [1.        , 0.96721311, 0.91428571, ..., 0.28571429, 0.59259259,
        0.        ]])

In [12]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_test

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,

In [13]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]


Create Neural Network Model

In [34]:
# Create model and add layers
model = Sequential()
model.add(Dense(4, input_dim=19, activation='sigmoid'))
#model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='softmax'))

In [35]:
#View model summary

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 4)                 80        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________


Train Model

In [36]:
#Compile model
from tensorflow.keras.optimizers import SGD

opt = SGD(lr=0.01)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
print(outcome.shape)

(1408,)


In [38]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
)

Epoch 1/100
33/33 [==============================] - 0s 2ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 2/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 3/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 4/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 5/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 6/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 7/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 8/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 9/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 10/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 11/

33/33 [==============================] - 0s 2ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 83/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 84/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 85/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 86/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 87/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 88/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 89/100
33/33 [==============================] - 0s 4ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 90/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 91/100
33/33 [==============================] - 0s 3ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 92/100


Evaluate Model

In [25]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - loss: 7.6246 - accuracy: 0.5000
Deep Neural Network - Loss: 7.624619007110596, Accuracy: 0.5


Predict World Series Champs

In [26]:
#Predict outcome
encoded_predictions = model.predict_classes(X_test_scaled[:37])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:37])}")

Predicted classes: ['NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL'
 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL'
 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL' 'NL']
Actual Labels: ['NL', 'AL', 'NL', 'NL', 'NL', 'NL', 'AL', 'AL', 'AL', 'AL', 'AL', 'AL', 'AL', 'NL', 'NL', 'NL', 'AL', 'NL', 'NL', 'AL', 'AL', 'AL', 'AL', 'NL', 'NL', 'AL', 'AL', 'AL', 'AL', 'AL', 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL']
